### define paths

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
NUM_THREADS = 16
# root_path = "/content/drive/MyDrive/llama-data/AraLlamaProject/"
root_path = "/content/drive/MyDrive/AraLlamaProject/"

### install some depencices and library

In [3]:
!nvidia-smi

Mon Sep  4 12:12:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install --quiet datasets sentencepiece argparse
!pip install --quiet git+https://github.com/huggingface/transformers@v4.28.1
!pip install --quiet datasets
!pip install --quiet sentencepiece
!pip install arabert
!pip install farasapy
!pip install evaluate
!pip install deepspeed
!pip install accelerate
!pip install bitsandbytes

In [5]:
from datasets import load_dataset
import tempfile
from tqdm import tqdm
import io
import pandas as pd
import sentencepiece as spm
from transformers import LlamaTokenizer
import os
from google.colab import drive
import os
from transformers import LlamaTokenizer
from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model
import sentencepiece as spm
import argparse
import re
import statistics
import math
import matplotlib.pyplot as plt
import numpy as np
from tokenizers import Tokenizer
from arabert import ArabertPreprocessor
from tokenizers.models import BPE, Unigram, WordLevel, WordPiece
from tokenizers.trainers import BpeTrainer, WordLevelTrainer, \
                                WordPieceTrainer, UnigramTrainer
from tokenizers.pre_tokenizers import Whitespace
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, Trainer, PreTrainedTokenizerFast
from transformers import Trainer, TrainingArguments, pipeline
from transformers import RobertaConfig, RobertaTokenizer, RobertaModel, RobertaForMaskedLM
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import RobertaConfig, RobertaTokenizer, RobertaModel, RobertaForMaskedLM
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import BertTokenizer
import tokenizers
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
import os
import json
from datasets import concatenate_datasets, load_dataset

[2023-09-04 12:14:26,704] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


# dataset preprocessing

In [6]:
dataset = load_dataset('nthngdy/oscar-small', 'unshuffled_deduplicated_ar')
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 408438
    })
})

In [7]:
dataset = dataset['train'] #.select(range(408438))
dataset

Dataset({
    features: ['id', 'text'],
    num_rows: 408438
})

In [8]:
import pandas as pd
import glob

# Get the file path of all txt files
file_path = glob.glob("/content/drive/MyDrive/AraLlamaProject/Dialects-Full-Text/*.txt")

# Create an empty list to store the dataframes
data_frames = []

# Loop through all the txt files
for file in file_path:
    # Read the txt file and create a dataframe
    df = pd.read_table(file, names=["text"])

    # Add the dataframe to the list
    data_frames.append(df)

# Concatenate the dataframes into one dataframe
df = pd.concat(data_frames)

# Set the column name to "text"
df.columns = ["text"]

# Print the dataframe
df
df =df.dropna()

In [11]:
from datasets import Dataset
dataset_1 = Dataset.from_pandas(df)
dataset_1
dataset_1 = dataset_1.rename_column("__index_level_0__", "id")

In [12]:
from datasets import concatenate_datasets, load_dataset
dataset = concatenate_datasets([dataset ,dataset_1])

In [13]:
# -----------------------------------------------------------------------------------------------------------
REGEXS = [
    r"(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%.\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%\+.~#?&//=])", #matches a URL
    r"@(https?|ftp)://(-\.)?([^\s/?\.#-]+\.?)+(/[^\s])?$@iS", #matches an email address
    r"http[s]?://[a-zA-Z0-9_\-./~\?=%&]+", #matches a shortened URL
    r"www[a-zA-Z0-9_\-?=%&/.~]+", #matches a website address
    r"[a-zA-Z]+\.com", #matches a domain name
    r"(?=http)[^\s]+", #matches any string that starts with "http"
    r"(?=www)[^\s]+", #matches any string that starts with "www"
    r"://", #matches the `://` part of a URL
    r"[\u4e00-\u9fa5]+", #matches any Unicode character from the Chinese, Japanese, or Korean character sets
    # r"[a-zA-Z]+", #matches any alphabetic character
    r"[\u0400-\u052f]+", #matches any Cyrillic character
    r"[\u3040-\u30ff]+", #matches any Japanese character
    r"^(.*?\|){3,}.*$" , #matches a string that contains at least 3 consecutive vertical bars (|)
    r"^(.*?\::){2,}.*$" , #matches a string that contains at least 2 consecutive colons (:)
    r"^(.*?\%){8,}.*$" , #matches a string that contains at least 8 consecutive percent signs (%)
    r'^[0-9]*$' , #matches a string of only digits
    r"^\w+$", #matches a string of only alphanumeric characters
    r"[\u0600-\u06FF]{15,}", #matches a string of Arabic characters that is at least 15 characters long
    # r"[^\u0600-\u06FF]" #matches any character that is not an Arabic character
]
# -----------------------------------------------------------------------------------------------------------

In [14]:
def preprocess_function(data):
  for regex in REGEXS:
    match = re.match(regex, data['text'])
    if(match):
      data['text'] = " "
  return data
def delete_condition(data):
  text = data["text"]
  return text != " "
dataset =dataset.map(preprocess_function, num_proc= NUM_THREADS)
dataset = dataset.filter(delete_condition, num_proc= NUM_THREADS)

Map (num_proc=16):   0%|          | 0/418540 [00:00<?, ? examples/s]

Filter (num_proc=16):   0%|          | 0/418540 [00:00<?, ? examples/s]

In [15]:
dataset[:10]

{'id': [0, 1, 2, 3, 4, 5, 6, 9, 11, 12],
 'text': ['مرحبا بك عزيز الزائر نتمنى لك أوقاتاً سعيدة معنا وأن نزداد شرفا بخدمتك ولا تنسى التسجيل معنا لتستفيد بكل جديد',
  'أهلا وسهلا بك زائرنا الكريم, أنت لم تقم بتسجيل الدخول بعد! يشرفنا أن تقوم بالدخول أو التسجيل إذا رغبت بالمشاركة في المنتدى',
  'نرحب بكل الزائرين ... ونتمى لكم قضاء وقت ممتع معنا ... يملأه الحب والود ... والاستفادة المتبادلة بيننا علميا وعمليا .....',
  'يسعدنا تسجيلكم معنا ومشاركتنا .... وشعارنا دوما (( نحب الخير لكل الناس .. مهما اختلفت الألوان والديانات والأجناس ))',
  'لي أربع شقيقات، أنا أكثرهن غنى، لكن لا أدري لماذا يأتي أقاربي لزيارة أخواتي بكثرة،وحينما يأتي موعد زيارتي لا يأتي سوى القليل،فهم يزورون أخواتي الأربع كل يوم،أما',
  'أنا أكثر أخواتي عطاءً لمن يأتيني.... لا أتهم أخواتي بالتقصير أبداً، ولكن الكل يعرف أني أكثرهن عطاءً...',
  'كثيرون ينصحون أقاربي بأن يأتوني، فلدي خير كثير وأعطي بكرم من يأتيني ومع ذلك يبتعدون عني، فلا حياة لمن تنادي ..',
  '[rtl]· عند التصوير يقوم الالة بمسح الاصول مرة واحده وتخزينها بالذاك

In [16]:
model_name="bert-base-arabert"
arabert_prep_0 = ArabertPreprocessor(
    model_name= model_name,
    keep_emojis = False,
    remove_html_markup = True,
    replace_urls_emails_mentions = True,
    strip_tashkeel = True,
    strip_tatweel = True,
    insert_white_spaces = True,
    remove_non_digit_repetition = True,
    replace_slash_with_dash = None,
    map_hindi_numbers_to_arabic = True,
    apply_farasa_segmentation = False
)
arabert_prep_1 = ArabertPreprocessor(
    model_name= model_name,
    keep_emojis = False,
    remove_html_markup = True,
    replace_urls_emails_mentions = True,
    strip_tashkeel = True,
    strip_tatweel = True,
    insert_white_spaces = True,
    remove_non_digit_repetition = True,
    replace_slash_with_dash = None,
    map_hindi_numbers_to_arabic = True,
    apply_farasa_segmentation = True
)
def preprocessing_withoutfarese(data):
  data['text'] = arabert_prep_0.preprocess(data['text'])
  return data
def preprocessing_withfarese(data):
  data['text'] = arabert_prep_1.preprocess(data['text'])
  return data

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [00:26<00:00, 8.94MiB/s]


[2023-09-04 12:23:37,967 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [17]:
dataset_nofarasa =dataset.map(preprocessing_withoutfarese, num_proc = NUM_THREADS)

Map (num_proc=16):   0%|          | 0/411191 [00:00<?, ? examples/s]

In [ ]:
dataset_withfarasa =dataset.map(preprocessing_withfarese)

Parameter 'function'=<function preprocessing_withfarese at 0x787cfed4b2e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/411191 [00:00<?, ? examples/s]

In [ ]:
dataset[:10]

{'id': [0, 1, 2, 3, 4, 5, 6, 9, 11, 12],
 'text': ['مرحبا بك عزيز الزائر نتمنى لك أوقاتاً سعيدة معنا وأن نزداد شرفا بخدمتك ولا تنسى التسجيل معنا لتستفيد بكل جديد',
  'أهلا وسهلا بك زائرنا الكريم, أنت لم تقم بتسجيل الدخول بعد! يشرفنا أن تقوم بالدخول أو التسجيل إذا رغبت بالمشاركة في المنتدى',
  'نرحب بكل الزائرين ... ونتمى لكم قضاء وقت ممتع معنا ... يملأه الحب والود ... والاستفادة المتبادلة بيننا علميا وعمليا .....',
  'يسعدنا تسجيلكم معنا ومشاركتنا .... وشعارنا دوما (( نحب الخير لكل الناس .. مهما اختلفت الألوان والديانات والأجناس ))',
  'لي أربع شقيقات، أنا أكثرهن غنى، لكن لا أدري لماذا يأتي أقاربي لزيارة أخواتي بكثرة،وحينما يأتي موعد زيارتي لا يأتي سوى القليل،فهم يزورون أخواتي الأربع كل يوم،أما',
  'أنا أكثر أخواتي عطاءً لمن يأتيني.... لا أتهم أخواتي بالتقصير أبداً، ولكن الكل يعرف أني أكثرهن عطاءً...',
  'كثيرون ينصحون أقاربي بأن يأتوني، فلدي خير كثير وأعطي بكرم من يأتيني ومع ذلك يبتعدون عني، فلا حياة لمن تنادي ..',
  '[rtl]· عند التصوير يقوم الالة بمسح الاصول مرة واحده وتخزينها بالذاك

In [ ]:
dataset_nofarasa[:10]

{'id': [0, 1, 2, 3, 4, 5, 6, 9, 11, 12],
 'text': ['مرحبا بك عزيز الزائر نتمنى لك أوقاتا سعيدة معنا وأن نزداد شرفا بخدمتك ولا تنسى التسجيل معنا لتستفيد بكل جديد',
  'أهلا وسهلا بك زائرنا الكريم , أنت لم تقم بتسجيل الدخول بعد ! يشرفنا أن تقوم بالدخول أو التسجيل إذا رغبت بالمشاركة في المنتدى',
  'نرحب بكل الزائرين . ونتمى لكم قضاء وقت ممتع معنا . يملأه الحب والود . والاستفادة المتبادلة بيننا علميا وعمليا .',
  'يسعدنا تسجيلكم معنا ومشاركتنا . وشعارنا دوما ( نحب الخير لكل الناس . مهما اختلفت الألوان والديانات والأجناس )',
  'لي أربع شقيقات ، أنا أكثرهن غنى ، لكن لا أدري لماذا يأتي أقاربي لزيارة أخواتي بكثرة ، وحينما يأتي موعد زيارتي لا يأتي سوى القليل ، فهم يزورون أخواتي الأربع كل يوم ، أما',
  'أنا أكثر أخواتي عطاء لمن يأتيني . لا أتهم أخواتي بالتقصير أبدا ، ولكن الكل يعرف أني أكثرهن عطاء .',
  'كثيرون ينصحون أقاربي بأن يأتوني ، فلدي خير كثير وأعطي بكرم من يأتيني ومع ذلك يبتعدون عني ، فلا حياة لمن تنادي .',
  '[rtl ] · عند التصوير يقوم الالة بمسح الاصول مرة واحده وتخزينها بالذاكره Scan O

In [ ]:
dataset_withfarasa[:10]

{'id': [178896,
  338296,
  338821,
  65149,
  387192,
  66125,
  295334,
  284005,
  227007,
  196469],
 'text': ['بوب مايرز ال+ مدير ال+ عام لغولدن ستيت رحب ب+ تجديد عقد دوران +ت قائل +ا : " في بعض ال+ أحيان لا تتفاوض ، أنا أحب أن أحصل على توقيع دوران +ت ب+ عقد ل+ 10 سنو +ات " .',
  'ال+ عجيب أن نوال ال+ سعداوي تعيب على كبار ال+ مثقف +ين في مصر و+ رؤساء مجالس إدار +ة ال+ صحف ال+ كبرى و+ ال+ كتاب أن +هم لم يقف +وا إلى جانب +ها مدافع +ين عما أسم +ت +ه بحري +ة ال+ فكر في مواجه +ة ال+ رد +ة ال+ ثقافي +ة و+ ال+ رجعي +ة ال+ ديني +ة . و+ ك+ أن +ها تبكي حق +ا سلب من +ها في سب ال+ دين متى شاء +ت . ال+ استهزاء ب+ قيم ال+ إسلام متى شاء +ت . حتى وصل ال+ أمر ب+ +ها إلي ال+ عيب في ال+ ذات ال+ إلهي +ة .',
  'ال+ فتر +ة التي تل +ت سقوط نظام صدام +ات ب+ ال+ إمكان أن نقول عن +ها ذهبي +ة ، خاص +ة فيما يتعلق ب+ ال+ جانب ال+ اقتصادي ، حيث قام نادي باريس ب+ إعفاء ال+ عراق ب+ ما يزيد عن 80 % من ال+ ديون ال+ مترتب +ة على نظام صدام ، ناهيك عن رفع ال+ حصار عن مليار +ات أخرى كان +ت محجوزة لدى بنوك أجنبي +ة ، و

In [ ]:
temp_file = tempfile.NamedTemporaryFile(mode='w+', delete=False)
file_dir_nofarasa = temp_file.name
text_list = []
for line in tqdm(dataset_nofarasa):
    text_list.append(line['text'] + '\n')

text = '\n'.join(text_list)
with io.StringIO(text) as f:
    temp_file.write(f.getvalue())
temp_file.close()

100%|██████████| 401312/401312 [00:13<00:00, 30595.12it/s]


In [ ]:
temp_file = tempfile.NamedTemporaryFile(mode='w+', delete=False)
file_dir_withfarasa= temp_file.name
text_list = []
for line in tqdm(dataset_withfarasa):
    text_list.append(line['text'] + '\n')

text = '\n'.join(text_list)
with io.StringIO(text) as f:
    temp_file.write(f.getvalue())
temp_file.close()

100%|██████████| 203/203 [00:00<00:00, 3122.30it/s]


# tokenization

In [ ]:
unk_token = "<unk>"  # token for unknown words
spl_tokens = ["<unk>", "<sep>", "<mask>", "<cls>", "<pad>"]  # special tokens

def prepare_tokenizer_trainer(alg , vocab_size):
    """
    Prepares the tokenizer and trainer with unknown & special tokens.
    """
    if alg == 'BPE':
        tokenizer = Tokenizer(BPE(unk_token = unk_token))
        trainer = BpeTrainer(special_tokens = spl_tokens , vocab_size=vocab_size)
    elif alg == 'UNI':
        tokenizer = Tokenizer(Unigram())
        trainer = UnigramTrainer(unk_token= unk_token, special_tokens = spl_tokens , vocab_size=vocab_size)
    elif alg == 'WPC':
        tokenizer = Tokenizer(WordPiece(unk_token = unk_token))
        trainer = WordPieceTrainer(special_tokens = spl_tokens , vocab_size=vocab_size)
    elif alg == "WLV":
        tokenizer = Tokenizer(WordLevel(unk_token = unk_token))
        trainer = WordLevelTrainer(special_tokens = spl_tokens,vocab_size=vocab_size)

    tokenizer.pre_tokenizer = Whitespace()
    return tokenizer, trainer
# ==================================================================================================================
def train_tokenizer(files, alg, vocab_size,dataset_type):
    tokenizer_path = root_path+"tokenizers/xlm-roberta-{}-tokenizer-{}-{}.json".format(alg , vocab_size,dataset_type)
    tokenizer, trainer = prepare_tokenizer_trainer(alg,vocab_size)
    tokenizer.train([files], trainer) # training the tokenzier
    tokenizer.save(tokenizer_path)
    tokenizer = Tokenizer.from_file(tokenizer_path)
    fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer, sep_token = "<sep>", unk_token = "<unk>", mask_token = "<mask>", cls_token = "<cls>", pad_token = "<pad>")

    return fast_tokenizer

# pretraining

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Davlan/xlm-roberta-base-finetuned-arabic")
print('No of parameters: ', model.num_parameters())

If you want to use `XLMRobertaLMHeadModel` as a standalone, add `is_decoder=True.`


No of parameters:  278295186


In [ ]:
def load_parameters(model , vocab,file_dir, tok,dataset_type):
# ===============================================================================
  tokenizer = train_tokenizer(file_dir, tok, vocab,dataset_type)
# ===============================================================================
  new_dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = file_dir,
    block_size = 128
  )

  args = TrainingArguments(
      output_dir="./output",
      num_train_epochs=1,
      adam_beta1=0.90,
      adam_beta2 =0.98,
      adam_epsilon = 1e-6,
      learning_rate=5e-5,
      per_device_train_batch_size=4,  # Adjust this based on your GPU memory
      gradient_accumulation_steps=1,   # Accumulate gradients over 4 batches
      # save_steps=1000,
      # warmup_ratio = 1e-2,
      fp16=True,

  )
  # ===============================================================================
  data_collator = DataCollatorForLanguageModeling(
      tokenizer=tokenizer, mlm=True, mlm_probability=0.15
  )
  # ===============================================================================
  trainer = Trainer(
      model=model,
      args=args,
      data_collator=data_collator,
      train_dataset=new_dataset,
  )
  return trainer
# ==============================================================================

### Train without using Farasa Preprocessing

In [ ]:
# vocab_size = [7000,16000,28000,44000,66000]
# tok_list = ["UNI", "BPE", "WPC", "WLV"]
vocab_size = [7000]
tok_list = ["UNI"]
# =============================================================================
for tok in tok_list:
  print("current tokenizer is {}\n".format(tok))
  for vocab in vocab_size:
    # =========================================================================
    trainer = load_parameters(model , vocab , file_dir_nofarasa, tok,"nofarasa")
    trainer.train()
    # ==========================================================================
    df = pd.DataFrame(trainer.state.log_history)
    df.to_excel(root_path + "Models/losses_{}_{}_{}.xlsx".format(tok ,vocab , "nofarasa" ))
    # ==========================================================================
    trainer.save_model(root_path+'Models/xlm-roberta-nofarasa-{}/{}'.format(vocab, tok))

current tokenizer is UNI



/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,6.992900


### train with farse

In [ ]:
vocab_size = [7000,16000,28000,44000,66000]
tok_list = ["BPE"]
# ==============================================================================
for tok in tok_list:
  print("current tokenizer is {}\n".format(tok))
  for vocab in vocab_size:
    # ==========================================================================
    trainer = load_parameters(model , vocab , file_dir_withfarasa, tok , "withfarasa")
    trainer.train()
    # ==========================================================================
    df = pd.DataFrame(trainer.state.log_history)
    df.to_excel(root_path+"Models/losses_{}_{}_{}.xlsx".format(tok ,vocab , "withfarasa" ))
    # ==========================================================================
    trainer.save_model(root_path+'Models/xlm-roberta-with-farasa-{}/{}'.format(vocab, tok))

### Train without using Farasa Preprocessing using deepspeed

In [ ]:
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '9994' # modify if RuntimeError: Address already in use
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"

In [ ]:
%%bash
cat <<'EOT' > ds_config_zero3.json
{
  "zero_optimization": {
     "stage": 2,
     "offload_optimizer": {
         "device": "cpu",
         "pin_memory": true
     },
     "allgather_partitions": true,
     "allgather_bucket_size": 2e8,
     "reduce_scatter": true,
     "reduce_bucket_size": 2e8,
     "overlap_comm": true,
     "contiguous_gradients": true
  },
    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto"
}


EOT

In [ ]:
def load_parameters(model , vocab,file_dir, tok,dataset_type):
# ===============================================================================
  tokenizer = train_tokenizer(file_dir, tok, vocab,dataset_type)
# ===============================================================================
  new_dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = file_dir,
    block_size = 128
  )

  args = TrainingArguments(
      output_dir="./output",
      num_train_epochs=10,
      adam_beta1=0.90,
      adam_beta2 =0.98,
      adam_epsilon = 1e-6,
      learning_rate=5e-5,
      per_device_train_batch_size=8,  # Adjust this based on your GPU memory
      gradient_accumulation_steps=22,   # Accumulate gradients over 4 batches
      save_steps=1000,
      warmup_ratio = 1e-2,
      fp16=True,
      deepspeed="./ds_config_zero3.json"

  )
  # ===============================================================================
  data_collator = DataCollatorForLanguageModeling(
      tokenizer=tokenizer, mlm=True, mlm_probability=0.15
  )
  # ===============================================================================
  trainer = Trainer(
      model=model,
      args=args,
      data_collator=data_collator,
      train_dataset=new_dataset,
  )
  return trainer
# ==============================================================================

In [ ]:
# vocab_size = [7000,16000,28000,44000,66000]
# tok_list = ["UNI", "BPE", "WPC", "WLV"]
vocab_size = [7000]
tok_list = ["UNI"]
# =============================================================================
for tok in tok_list:
  print("current tokenizer is {}\n".format(tok))
  for vocab in vocab_size:
    # =========================================================================
    trainer = load_parameters(model , vocab , file_dir_nofarasa, tok,"nofarasa")
    trainer.train()
    # ==========================================================================
    df = pd.DataFrame(trainer.state.log_history)
    df.to_excel(root_path + "Models/losses_{}_{}_{}.xlsx".format(tok ,vocab , "nofarasa" ))
    # ==========================================================================
    trainer.save_model(root_path+'Models/xlm-roberta-nofarasa-{}/{}'.format(vocab, tok))

### Train without using Farasa Preprocessing using LORA

In [ ]:
!pip install transformers accelerate evaluate datasets peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.2 MB/s eta 0:00:00


In [ ]:
from peft import LoraConfig, get_peft_model


# print_trainable_parameters(lora_model)

In [ ]:
def load_parameters(model , vocab,file_dir, tok,dataset_type):
# ===============================================================================
  tokenizer = train_tokenizer(file_dir, tok, vocab,dataset_type)
# ===============================================================================
  new_dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = file_dir,
    block_size = 128
  )
  lora_config = {
    "fp16": True,
    "pipeline_parallel_size": 2,
    "gradient_accumulation_steps": 22,
  }
  args = TrainingArguments(
      output_dir="./output",
      num_train_epochs=10,
      adam_beta1=0.90,
      adam_beta2 =0.98,
      adam_epsilon = 1e-6,
      learning_rate=5e-5,
      per_device_train_batch_size=8,  # Adjust this based on your GPU memory
      gradient_accumulation_steps=22,   # Accumulate gradients over 4 batches
      save_steps=1000,
      warmup_ratio = 1e-2,
      fp16=True

  )
  # ===============================================================================
  data_collator = DataCollatorForLanguageModeling(
      tokenizer=tokenizer, mlm=True, mlm_probability=0.15
  )
  # ===============================================================================
  trainer = Trainer(
      model=model,
      args=args,
      data_collator=data_collator,
      train_dataset=new_dataset,
      #lora_config=lora_config,
      accelerator="auto"
  )
  return trainer
# ==============================================================================

In [ ]:
# vocab_size = [7000,16000,28000,44000,66000]
# tok_list = ["UNI", "BPE", "WPC", "WLV"]
vocab_size = [7000]
tok_list = ["UNI"]
# =============================================================================
for tok in tok_list:
  print("current tokenizer is {}\n".format(tok))
  for vocab in vocab_size:
    # =========================================================================
    trainer = load_parameters(model , vocab , file_dir_nofarasa, tok, "nofarasa")
    trainer.train()
    # ==========================================================================
    df = pd.DataFrame(trainer.state.log_history)
    df.to_excel(root_path + "Models/losses_{}_{}_{}.xlsx".format(tok ,vocab , "nofarasa" ))
    # ==========================================================================
    trainer.save_model(root_path+'Models/xlm-roberta-nofarasa-{}/{}'.format(vocab, tok))

current tokenizer is UNI



/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


TypeError: ignored